In [38]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import Counter
from itertools import combinations_with_replacement as combos
from itertools import permutations as perms
from tensorflow.keras import layers, Model
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import coverage_error, f1_score, label_ranking_average_precision_score, average_precision_score
from sklearn.model_selection import train_test_split, GridSearchCV
from category_encoders import OneHotEncoder
from tensorflow.data import Dataset


In [3]:
def is_three_pair(choice):
    choice = sorted(choice)
    return (len(choice) == 6 and choice[0] == choice[1] and
            choice[2] == choice[3] and choice[4] == choice[5])


def is_straight(choice):
    return sorted(choice) == list(range(1, 7))


def score_all():
    return [1.] * 6

scoring_rules = [[100, 200, 1000, 2000, 4000, 5000],
                 [0, 0, 200, 400, 800, 5000],
                 [0, 0, 300, 600, 1200, 5000],
                 [0, 0, 400, 800, 1600, 5000],
                 [50, 100, 500, 1000, 2000, 5000],
                 [0, 0, 600, 1200, 2400, 5000]
                 ]

def choose_dice(roll):
    """Choose dice according to scoring rules. Boop Beep."""
    counts = Counter(roll)
    if is_three_pair(roll) and (sum(scoring_rules[die - 1][count - 1] for die, count in counts.items()) < 1500):
        choice = score_all()
    elif is_straight(roll):
        choice = score_all()
    else:
        picks = set()
        for die, count in counts.items():
            if scoring_rules[die - 1][count - 1] > 0:
                picks.add(die)
        choice = [0.] * 6
        for i, x in enumerate(roll):
            if x in picks:
                choice[i] = 1.
    return np.array(choice)


In [4]:
def make_some_features(numbers, clip):
    features = set()
    combinations = (combo for combo in combos(numbers, 6))
    for i, comb in enumerate(combinations):
        if i % clip == 0:  # Keeping size reasonable
            for perm in perms(comb):
                features.add(perm)
    return features


In [5]:
def make_special_features():
    special_features = set()
    for _ in range(7500):
        half = [np.random.randint(1, 6) for _ in range(3)]
        half += half
        for perm in perms(half):
            special_features.add(perm)

    for perm in perms([1, 2, 3, 4, 5, 6]):
        special_features.add(perm)
    
    return special_features


In [6]:
features = make_some_features(list(range(1, 7)), 2)
# special_features = make_special_features()

# all_features = [np.array(feature) for feature in special_features]
# print(len(all_features))
all_features = [np.array(feature) for feature in features]

all_labels = [choose_dice(feature) for feature in all_features]
all_features = np.array(all_features)
all_labels = np.array(all_labels)
len(all_features), len(all_labels)

(23114, 23114)

In [7]:
def create_dataset(features, labels):
    data = {str(i): features[:,i] for i in range(6)}
    dataset = pd.DataFrame(data)
    label = {'{}_l'.format(i): labels[:,i] for i in range(6)}
    label_df = pd.DataFrame(label)
    df = pd.concat([dataset, label_df], axis=1, sort=False)
    return df


In [8]:
df = create_dataset(all_features, all_labels)

In [9]:
df.sample(10)

,0,1,2,3,4,5,0_l,1_l,2_l,3_l,4_l,5_l
10038,2,5,2,5,3,4,0.0,1.0,0.0,1.0,0.0,0.0
8673,2,5,3,4,6,6,0.0,1.0,0.0,0.0,0.0,0.0
5917,2,6,6,3,6,6,0.0,1.0,1.0,0.0,1.0,1.0
11527,6,4,6,1,6,2,1.0,0.0,1.0,1.0,1.0,0.0
15946,4,4,6,4,3,6,1.0,1.0,0.0,1.0,0.0,0.0
23060,5,5,1,6,2,5,1.0,1.0,1.0,0.0,0.0,1.0
21694,2,3,5,1,6,5,0.0,0.0,1.0,1.0,0.0,1.0
16949,4,6,2,1,6,3,0.0,0.0,0.0,1.0,0.0,0.0
22006,4,4,3,2,3,4,1.0,1.0,0.0,0.0,0.0,1.0
4371,2,1,6,4,6,4,0.0,1.0,0.0,0.0,0.0,0.0


In [10]:
X = df[['0', '1', '2', '3', '4', '5']]
y = df[['0_l', '1_l', '2_l', '3_l', '4_l', '5_l']]

In [11]:
def train_val_test_split(X, 
                         y,
                         train_size=.8,
                         val_size=.1,
                         test_size=.1,
                         random_state=42,
                         shuffle=True):
    
    assert train_size + val_size + test_size == 1
        

    X_trainval, X_test, y_trainval, y_test = train_test_split(X,
                                                              y,
                                                              test_size=test_size,
                                                              random_state=random_state,
                                                              shuffle=shuffle,
                                                              stratify=y
                                                             )
    X_train, X_val, y_train, y_val = train_test_split(X_trainval,
                                                      y_trainval,
                                                      test_size=val_size / (train_size + val_size),
                                                      random_state=random_state,
                                                      shuffle=shuffle,
                                                      stratify=y_trainval
                                                     )
    return X_train, X_val, X_test, y_train, y_val, y_test


In [12]:
X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(X, y)

In [13]:
X_train.shape, y_train.shape

((18490, 6), (18490, 6))

In [14]:
X_val.shape, y_val.shape

((2312, 6), (2312, 6))

In [15]:
X_test.shape, y_test.shape

((2312, 6), (2312, 6))

In [16]:
model = RandomForestClassifier(n_estimators=1000,
                               max_depth=20,
                               min_samples_split=3,
                               n_jobs=-1,
                               )

In [17]:
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=3,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [18]:
model.score(X_val, y_val)

0.7469723183391004

In [90]:
def test_model_pred(model, threshold=.422, samples=25):
    start = np.random.randint(0, 1392)
    print(start)
    stop = start + samples
    for test in all_features[start:stop]:
        print(test)
        true = choose_dice(test)
        print(true)
        pred = model.predict([list(test)])[0]
        print(pred)
        pred_proba = np.array([round(y[0][1], 3) for y in model.predict_proba([list(test)])])
        print(pred_proba)
        pred_thresh = (pred_proba > threshold).astype(int)
        print(pred_thresh)
        result = 'Nailed it' if list(true) == list(pred) else 'Nuts'
        print(result)
        result = 'Nailed it' if list(true) == list(pred_thresh) else 'Nuts'
        print(result)
        print

In [60]:
test_model_pred(model)

954
[2 3 2 5 2 5]
[1. 0. 1. 1. 1. 1.]
[1. 0. 1. 1. 1. 1.]
Nailed it

[1 2 2 3 1 4]
[1. 0. 0. 0. 1. 0.]
[1. 0. 0. 0. 1. 0.]
Nailed it

[2 1 6 6 3 1]
[0. 1. 0. 0. 0. 1.]
[0. 1. 0. 0. 0. 1.]
Nailed it

[6 4 1 2 1 6]
[0. 0. 1. 0. 1. 0.]
[0. 0. 1. 0. 1. 0.]
Nailed it

[1 5 5 6 4 2]
[1. 1. 1. 0. 0. 0.]
[1. 1. 1. 0. 0. 0.]
Nailed it

[3 2 2 2 1 4]
[0. 1. 1. 1. 1. 0.]
[0. 1. 1. 1. 1. 0.]
Nailed it

[1 5 2 3 2 4]
[1. 1. 0. 0. 0. 0.]
[1. 1. 0. 0. 0. 0.]
Nailed it

[2 1 1 6 4 6]
[0. 1. 1. 0. 0. 0.]
[0. 1. 1. 0. 0. 0.]
Nailed it

[2 4 5 1 3 2]
[0. 0. 1. 1. 0. 0.]
[0. 0. 1. 1. 0. 0.]
Nailed it

[5 4 2 1 2 3]
[1. 0. 0. 1. 0. 0.]
[1. 0. 0. 1. 0. 0.]
Nailed it

[6 4 4 6 2 2]
[1. 1. 1. 1. 1. 1.]
[0. 1. 1. 0. 1. 1.]
Nuts

[6 1 5 6 5 5]
[0. 1. 1. 0. 1. 1.]
[0. 1. 1. 0. 1. 1.]
Nailed it

[6 2 2 2 3 3]
[0. 1. 1. 1. 0. 0.]
[0. 1. 1. 1. 0. 0.]
Nailed it

[4 1 3 1 5 3]
[0. 1. 0. 1. 1. 0.]
[0. 1. 0. 1. 1. 0.]
Nailed it

[4 5 3 3 1 6]
[0. 1. 0. 0. 1. 0.]
[0. 1. 0. 0. 1. 0.]
Nailed it

[2 1 1 4 2 2]
[1. 1. 1. 0.

In [21]:
extra = ExtraTreesClassifier(bootstrap=True,
                             max_depth=25,
                             n_jobs=-1,
                             min_samples_split=3,
                             n_estimators=400)
extra.fit(X_train, y_train)
extra.score(X_val, y_val)

0.8019031141868512

In [22]:
test_model_pred(extra)

1339
[2 3 5 1 4 4]
[0. 0. 1. 1. 0. 0.]
[0. 0. 1. 1. 0. 0.]
[5 5 3 6 5 6]
[1. 1. 0. 0. 1. 0.]
[1. 1. 0. 0. 1. 0.]
[3 2 5 4 3 1]
[0. 0. 1. 0. 0. 1.]
[0. 0. 1. 0. 0. 1.]
[3 2 4 4 5 1]
[0. 0. 0. 0. 1. 1.]
[0. 0. 0. 0. 1. 1.]
[6 5 1 6 4 6]
[1. 1. 1. 1. 0. 1.]
[1. 1. 1. 1. 0. 1.]
[2 2 4 5 3 3]
[0. 0. 0. 1. 0. 0.]
[0. 0. 0. 1. 0. 0.]
[4 2 4 3 4 3]
[1. 0. 1. 0. 1. 0.]
[1. 0. 1. 0. 1. 0.]
[1 1 4 2 4 5]
[1. 1. 0. 0. 0. 1.]
[1. 1. 0. 0. 0. 1.]
[6 3 6 2 6 6]
[1. 0. 1. 0. 1. 1.]
[1. 0. 1. 0. 1. 1.]
[3 1 6 5 5 2]
[0. 1. 0. 1. 1. 0.]
[0. 1. 0. 1. 1. 0.]
[3 3 6 4 5 6]
[0. 0. 0. 0. 1. 0.]
[0. 0. 0. 0. 1. 0.]
[5 6 3 5 3 6]
[1. 1. 1. 1. 1. 1.]
[1. 0. 0. 1. 0. 0.]
[6 1 1 3 1 4]
[0. 1. 1. 0. 1. 0.]
[0. 1. 1. 0. 1. 0.]
[4 2 2 5 4 5]
[1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1.]
[2 5 3 2 5 4]
[0. 1. 0. 0. 1. 0.]
[0. 1. 0. 0. 1. 0.]
[1 6 1 5 6 1]
[1. 0. 1. 1. 0. 1.]
[1. 0. 1. 1. 0. 1.]
[3 5 5 5 4 3]
[0. 1. 1. 1. 0. 0.]
[0. 1. 1. 1. 0. 0.]
[6 5 5 1 4 1]
[0. 1. 1. 1. 0. 1.]
[0. 1. 1. 1. 0. 1.]
[2 5 2 4 5 3]
[0. 1. 0.

In [23]:
params = {'n_estimators': [1000, 1500, 2000, 3000]}

grid = GridSearchCV(extra,
                    param_grid=params,
                    scoring='average_precision',
                    n_jobs=-1,
                    cv=5,
                    verbose=1)
grid.fit(X_train, y_train)
grid.best_params_, grid.best_score_

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


KeyboardInterrupt: 

In [ ]:
params = {'max_depth': [15, 20, 25, 30]}
grid = GridSearchCV(extra,
                    param_grid=params,
                    scoring='average_precision',
                    n_jobs=-1,
                    cv=5,
                    verbose=1)
grid.fit(X_train, y_train)
grid.best_params_, grid.best_score_

In [ ]:
params = {'min_samples_split': [5, 6, 7, 8]}
grid = GridSearchCV(extra,
                    param_grid=params,
                    scoring='average_precision',
                    n_jobs=-1,
                    cv=5,
                    verbose=1)
grid.fit(X_train, y_train)
grid.best_params_, grid.best_score_

In [ ]:
params = {'oob_score': [True, False]}
grid = GridSearchCV(extra,
                    param_grid=params,
                    scoring='average_precision',
                    n_jobs=-1,
                    cv=5,
                    verbose=1)
grid.fit(X_train, y_train)
grid.best_params_, grid.best_score_

In [24]:
params = {'min_samples_split': [4, 5, 6],
          'max_depth': [27, 30, 33],
          'oob_score': [True],
          'n_estimators': [2000]}
grid = GridSearchCV(extra,
                    param_grid=params,
                    scoring='average_precision',
                    n_jobs=-1,
                    cv=5,
                    verbose=1)
grid.fit(X_train, y_train)
grid.best_params_, grid.best_score_

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed: 43.8min finished


({'max_depth': 30,
  'min_samples_split': 6,
  'n_estimators': 2000,
  'oob_score': True},
 0.9761724583282346)

In [ ]:
# params = {'min_samples_split': [3, 4, 5],
#           'max_depth': [26, 27, 28],
#           'oob_score': [True],
#           'n_estimators': [2000]}
# grid = GridSearchCV(extra,
#                     param_grid=params,
#                     scoring='average_precision',
#                     n_jobs=-1,
#                     cv=5,
#                     verbose=1)
# grid.fit(X_train, y_train)
# grid.best_params_, grid.best_score_

In [25]:
params = {'n_estimators': [1250, 1500, 1750, 2000, 2250, 2500]}
grid = GridSearchCV(grid.best_estimator_,
                    param_grid=params,
                    scoring='average_precision',
                    n_jobs=-1,
                    cv=5,
                    verbose=1)
grid.fit(X_train, y_train)
grid.best_params_, grid.best_score_

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 25.6min finished


({'n_estimators': 1500}, 0.9762026089530731)

In [26]:
best = grid.best_estimator_

In [27]:
best.score(X_val, y_val)

0.828719723183391

In [94]:
test_model_pred(best, threshold=.45, samples=40)

558
[4 3 1 1 3 5]
[0. 0. 1. 1. 0. 1.]
[0. 0. 1. 1. 0. 1.]
[0.176 0.185 0.912 0.925 0.192 0.891]
[0 0 1 1 0 1]
Nailed it
Nailed it

[6 2 3 3 6 5]
[0. 0. 0. 0. 0. 1.]
[0. 0. 0. 0. 0. 1.]
[0.179 0.169 0.203 0.198 0.197 0.916]
[0 0 0 0 0 1]
Nailed it
Nailed it

[1 5 4 5 5 6]
[1. 1. 0. 1. 1. 0.]
[1. 1. 0. 1. 1. 0.]
[0.869 0.789 0.145 0.829 0.771 0.122]
[1 1 0 1 1 0]
Nailed it
Nailed it

[1 6 6 6 2 5]
[1. 1. 1. 1. 0. 1.]
[1. 1. 1. 1. 0. 1.]
[0.811 0.695 0.701 0.706 0.159 0.769]
[1 1 1 1 0 1]
Nailed it
Nailed it

[6 6 1 1 6 5]
[1. 1. 1. 1. 1. 1.]
[1. 1. 1. 1. 1. 1.]
[0.78  0.758 0.849 0.822 0.769 0.784]
[1 1 1 1 1 1]
Nailed it
Nailed it

[1 3 5 2 2 5]
[1. 0. 1. 0. 0. 1.]
[1. 0. 1. 0. 0. 1.]
[0.952 0.165 0.848 0.198 0.219 0.846]
[1 0 1 0 0 1]
Nailed it
Nailed it

[5 1 1 1 3 6]
[1. 1. 1. 1. 0. 0.]
[1. 1. 1. 1. 0. 0.]
[0.755 0.915 0.907 0.925 0.163 0.107]
[1 1 1 1 0 0]
Nailed it
Nailed it

[2 6 3 6 2 6]
[0. 1. 0. 1. 0. 1.]
[0. 1. 0. 1. 0. 1.]
[0.279 0.8   0.254 0.803 0.283 0.811]
[0 1 0 1 0 1]
N

In [29]:
y_pred = [best.predict([test])[0] for test in X_test.values]

In [30]:
y_pred = np.array(y_pred)

In [31]:
coverage_error(y_pred, y_test)

3.04022491349481

In [32]:
f1_score(y_test, y_pred, average= 'samples')

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/metrics/classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true labels.
  'recall', 'true', average, warn_for)


0.9060103943538199

In [33]:
y_pred.shape, y_test.shape

((2312, 6), (2312, 6))

In [34]:
y_test.sum()

0_l    1169.0
1_l    1170.0
2_l    1168.0
3_l    1169.0
4_l    1166.0
5_l    1170.0
dtype: float64

In [35]:
y_test.sum().sum() / len(y_test)

3.0328719723183393

In [36]:
label_ranking_average_precision_score(y_test, y_pred)

0.9749525422914274

In [39]:
average_precision_score(y_test, y_pred)

0.9138463830419251